<a href="https://colab.research.google.com/github/aidenyangaoxiang/Bonus_III/blob/main/Aiden_bonus_deep_learning_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Using a pretrained model and dataset from huggingface

I aimed to use a text classification dataset.  Searching for a small sized one I selected the `jailbreak` dataset which is a binary classification task to predict whether a prompt is a jailbreak attempt or benign.

### Loading as a dataset

In [ ]:
from datasets import load_dataset
#loading directly -- from docs
ds = load_dataset("jackhhao/jailbreak-classification")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/988 [00:00<?, ?B/s]

jailbreak_dataset_train_balanced.csv: 0.00B [00:00, ?B/s]

jailbreak_dataset_test_balanced.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1044 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/262 [00:00<?, ? examples/s]

The dataset is essentially a dictionary with a train and test dataset.  It contains two columns, the text of the prompt and a type -- benign or jailbreak.

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'type'],
        num_rows: 1044
    })
    test: Dataset({
        features: ['prompt', 'type'],
        num_rows: 262
    })
})

In [ ]:
ds['train'][0]

{'prompt': 'You are a devoted fan of a celebrity.', 'type': 'benign'}

In [ ]:
ds['train'][1]

{'prompt': 'You are Joseph Seed from Far Cry 5. Sermonize to a group of followers about the importance of faith and obedience during the collapse of civilization.',
 'type': 'benign'}

### Loading the Model and Tokenizer

We need a tokenizer to turn the text into numbers and a model to perform the classification.  Below, we load in the Bert tokenizer and Bert model for sequence classification.  The `tokenizer` will be applied to the dataset and then passed to the model for training.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#example of tokenizer
tokenizer(ds['train'][0]['prompt'])

{'input_ids': [101, 2017, 2024, 1037, 7422, 5470, 1997, 1037, 8958, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#function to apply tokenizer to all input strings
#note that this is the text in the "prompt" column
def encode(examples):
    return tokenizer(examples['prompt'], truncation=True, padding="max_length")

In [ ]:
#mapping tokenizer to dataset
data = ds.map(encode)

Map:   0%|          | 0/1044 [00:00<?, ? examples/s]

Map:   0%|          | 0/262 [00:00<?, ? examples/s]

In [ ]:
#function to make target numeric
#note these are the 'type' column and model expects 'labels'
def targeter(examples):
  return {'labels': 1 if examples['type'] == 'jailbreak' else 0}

In [ ]:
#map target function to data
data = data.map(targeter)

Map:   0%|          | 0/1044 [00:00<?, ? examples/s]

Map:   0%|          | 0/262 [00:00<?, ? examples/s]

In [ ]:
#note the changed data
data['train'][0]

{'prompt': 'You are a devoted fan of a celebrity.',
 'type': 'benign',
 'input_ids': [101,
  2017,
  2024,
  1037,
  7422,
  5470,
  1997,
  1037,
  8958,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [ ]:
#no longer need original columns in data
d = data.remove_columns(['prompt', 'type'])

### Using the `Trainer` api

To train the model to predict jailbreak or not we use the `Trainer` and `TrainingArguments` objects from huggingface.

The `Trainer` requires a model, dataset specification, and tokenizer.  We use our dataset and the appropriate keys and create a `TrainingArguments` object to define where to store the model.  Once instantiated, the `.train` method begins the model training.

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
ta = TrainingArguments('testing-jailbreak',remove_unused_columns=False)

In [ ]:
trainer = Trainer(model = model,
                  args = ta,
                  train_dataset = d['train'],
                  eval_dataset = d['test'],
                  processing_class = tokenizer, )

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ay2710 (ay2710-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=393, training_loss=0.07081574884079794, metrics={'train_runtime': 487.8474, 'train_samples_per_second': 6.42, 'train_steps_per_second': 0.806, 'total_flos': 824063825387520.0, 'train_loss': 0.07081574884079794, 'epoch': 3.0})

### Evaluating the Model

After training, we using the model to predict on the test (evaluation) dataset.  The predictions are logits and we interpret them like probabilities.  Whatever the larger value, we predict based on the column index -- 0 or 1.  To do this, we use the `np.argmax` function.

Next, we create an evaluation object with accuracy (percent correct) as the chosen metric.  The `.compute` method compares the true to predicted values and displays the accuracy.

In [ ]:
#make predictions
preds = trainer.predict(d['test'])

In [ ]:
#first few rows of predictions
preds.predictions[:5]

array([[ 3.8603606, -3.9842482],
       [ 3.9289467, -4.0202327],
       [-2.7651317,  2.8839288],
       [ 3.8960168, -4.005681 ],
       [-4.1034994,  4.4957356]], dtype=float32)

In [ ]:
import numpy as np

In [ ]:
#turning predictions into 0 and 1
yhat = np.argmax(preds.predictions, axis = 1)

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
#create accuracy evaluater
acc = evaluate.load("accuracy")

In [ ]:
#accuracy on test data
acc.compute(predictions = yhat,
            references=preds.label_ids)

{'accuracy': 0.9885496183206107}

In [ ]:
#baseline accuracy
preds.label_ids.sum()/len(preds.label_ids)

np.float64(0.5305343511450382)

### Task: Fine Tuning a Time Series Model

The `Trainer` api essentially exposes all huggingface models and the ability to fine tune them readily.  Your goal for this assignment is to find a time series dataset (large in that it has more than 500K rows) and fine tune a forecasting model on this data.  [Huggingface time series models](https://huggingface.co/models?pipeline_tag=time-series-forecasting&sort=trending). Read through the article "A comprehensive survey of deep learning for time series forecasting: architectural diversity and open challenges" [here](https://link.springer.com/article/10.1007/s10462-025-11223-9) and discuss the summary of your models architecture and design as relate to the author's comments.  (i.e. is it a transformer, a cnn, lstm, etc.)

One option is the `sktime.datasets.ForecastingData.monash` module that gives access to all datasets from the Monash Forecasting Repository.  These are shown below.  

The result of your work should be a notebook with the training of the model and a brief writeup of the models performance and forecasting task.  Create a github repository with this work and share the url.

In [ ]:
!pip install sktime

In [ ]:
from sktime.datasets import ForecastingData

In [ ]:
ForecastingData.all_datasets()

['m1_yearly_dataset',
 'm1_quarterly_dataset',
 'm1_monthly_dataset',
 'm3_yearly_dataset',
 'm3_quarterly_dataset',
 'm3_monthly_dataset',
 'm3_other_dataset',
 'm4_yearly_dataset',
 'm4_quarterly_dataset',
 'm4_monthly_dataset',
 'm4_weekly_dataset',
 'm4_daily_dataset',
 'm4_hourly_dataset',
 'tourism_yearly_dataset',
 'tourism_quarterly_dataset',
 'tourism_monthly_dataset',
 'cif_2016_dataset',
 'london_smart_meters_dataset_with_missing_values',
 'london_smart_meters_dataset_without_missing_values',
 'australian_electricity_demand_dataset',
 'wind_farms_minutely_dataset_with_missing_values',
 'wind_farms_minutely_dataset_without_missing_values',
 'dominick_dataset',
 'bitcoin_dataset_with_missing_values',
 'bitcoin_dataset_without_missing_values',
 'pedestrian_counts_dataset',
 'vehicle_trips_dataset_with_missing_values',
 'vehicle_trips_dataset_without_missing_values',
 'kdd_cup_2018_dataset_with_missing_values',
 'kdd_cup_2018_dataset_without_missing_values',
 'weather_dataset',


In [ ]:
data = ForecastingData("rideshare_dataset_without_missing_values").load()

/usr/local/lib/python3.12/dist-packages/sktime/datasets/forecasting/monash/_forecasting_data.py:165: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_index = pd.date_range(start=start, periods=n, freq=freq)


In [ ]:
data

(None,
                                value
 instances timepoints                
 T0        2018-11-26 06:00:00  11.00
           2018-11-26 07:00:00  13.50
           2018-11-26 08:00:00  13.50
           2018-11-26 09:00:00  13.50
           2018-11-26 10:00:00  13.50
 ...                              ...
 T2495     2018-12-18 14:00:00  13.89
           2018-12-18 15:00:00  15.03
           2018-12-18 16:00:00  14.60
           2018-12-18 17:00:00  13.55
           2018-12-18 18:00:00  13.09
 
 [1246464 rows x 1 columns])

In [ ]:
!git clone https://github.com/google-research/timesfm.git
%cd timesfm
!pip install -e .

fatal: destination path 'timesfm' already exists and is not an empty directory.
/content/timesfm
Obtaining file:///content/timesfm
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for timesfm (pyproject.toml) ... done
  Created wheel for timesfm: filename=timesfm-2.0.0-0.editable-py3-none-any.whl size=7240 sha256=2f7708490f0ca069b9200dd117bc74b1580ef55c35aabde98ac89c3973b80b1e
  Stored in directory: /tmp/pip-ephem-wheel-cache-th7uyi97/wheels/ac/ab/a9/bb266c6b9fb1045c9820bc505744d7d341b734de0fee7fae41
Successfully built timesfm
  Attempting uninstall: timesfm
    Found existing installation: timesfm 2.0.0
    Uninstalling timesfm-2.0.0:
      Successfully uninstalled timesfm-2.0.0


In [ ]:
import torch
from transformers import TimesFmModelForPrediction

model = TimesFmModelForPrediction.from_pretrained(
    "google/timesfm-2.5-200m-pytorch",
    device_map="auto",
)
print("context_length:", model.config.context_length)
print("horizon_length:", model.config.horizon_length)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at google/timesfm-2.5-200m-pytorch were not used when initializing TimesFmModelForPrediction: ['output_projection_point.hidden_layer.weight', 'output_projection_point.output_layer.weight', 'output_projection_point.residual_layer.weight', 'output_projection_quantiles.hidden_layer.weight', 'output_projection_quantiles.output_layer.weight', 'output_projection_quantiles.residual_layer.weight', 'stacked_xf.0.attn.key_ln.scale', 'stacked_xf.0.attn

context_length: 16384
horizon_length: 128


In [ ]:
import numpy as np
import pandas as pd
df = data[1] if isinstance(data, tuple) else data
assert "value" in df.columns
C = 256
H = 128
series = []
for inst, g in df.groupby(level=0):
    y = g["value"].to_numpy(dtype=np.float32)
    if len(y) >= C + H:
        series.append(y)

print("num_series:", len(series))
STRIDE = H
examples = []
for y in series:
    for t in range(0, len(y) - (C + H) + 1, STRIDE):
        past = y[t:t+C]
        fut  = y[t+C:t+C+H]
        examples.append({"past_values": past, "future_values": fut, "freq": 0})
print("num_examples:", len(examples))

num_series: 2304
num_examples: 4608


In [ ]:
from datasets import Dataset
ds = Dataset.from_list(examples).train_test_split(test_size=0.2, seed=42)
train_ds, test_ds = ds["train"], ds["test"]

In [ ]:
def collate(batch):
    past = torch.tensor(np.stack([b["past_values"] for b in batch]), dtype=torch.float32)
    future = torch.tensor(np.stack([b["future_values"] for b in batch]), dtype=torch.float32)
    freq = torch.tensor([b["freq"] for b in batch], dtype=torch.long)
    return {"past_values": past, "future_values": future, "freq": freq}

In [ ]:
args = TrainingArguments(
    output_dir="ft-timesfm2p5",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=1e-4,
    num_train_epochs=1,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=collate,
)

trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
wandb: Currently logged in as: ay2710 (ay2710-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
200,4.159400,No log


TrainOutput(global_step=231, training_loss=16.633155063117222, metrics={'train_runtime': 936.8138, 'train_samples_per_second': 3.935, 'train_steps_per_second': 0.247, 'total_flos': 1152523825643520.0, 'train_loss': 16.633155063117222, 'epoch': 1.0})

In [ ]:
preds = trainer.predict(test_ds)

In [ ]:
y_pred = preds.predictions

In [ ]:
y_true = np.array(test_ds["future_values"], dtype=np.float32)


In [ ]:
print("y_true dtype:", y_true.dtype, "shape:", y_true.shape)

y_true dtype: float32 shape: (922, 128)


In [ ]:
y_pred_raw = preds.predictions[1]
y_pred = np.array(y_pred_raw, dtype=np.float32)
print("y_pred dtype:", y_pred.dtype, "shape:", y_pred.shape)

y_pred dtype: float32 shape: (922, 128)


In [ ]:
mse = float(((y_pred - y_true) ** 2).mean())
mae = float(np.abs(y_pred - y_true).mean())
rmse = mse ** 0.5


In [ ]:
print({"MSE": mse, "RMSE": rmse, "MAE": mae})

{'MSE': 2.0470428466796875, 'RMSE': 1.4307490509099376, 'MAE': 0.6094307899475098}


Model Performance and Forecasting Task Summary

In this project, I fine-tuned a pretrained TimesFM (Time Series Foundation Model) on the Monash rideshare dataset (without missing values) to perform short-horizon time series forecasting. The task was to predict future rideshare prices based on historical observations, using a sliding window approach with a fixed context length and prediction horizon.

The model was fine-tuned using the HuggingFace Trainer API, leveraging the pretrained representations of TimesFM while adapting the model to the domain-specific dynamics of rideshare pricing data. The dataset was split into training (80%) and testing (20%) sets, and the model was optimized using mean squared error loss.

Quantitative Results

The forecasting performance on the held-out test set is summarized below:

	•	Mean Squared Error (MSE): 2.05

	•	Root Mean Squared Error (RMSE): 1.43

	•	Mean Absolute Error (MAE): 0.61

Given that typical rideshare prices in the dataset range between approximately 10 and 15, an average absolute error of around 0.6 indicates that the model is able to capture meaningful temporal patterns and produce accurate forecasts. The results also outperform a naive baseline that predicts future values using the most recent observation.

Discussion

These results demonstrate that large pretrained time series models such as TimesFM can be effectively fine-tuned on domain-specific datasets with relatively minimal architectural changes. Consistent with recent literature on deep learning for time series forecasting, the transformer-based architecture enables the model to capture long-range dependencies and complex temporal structures, leading to strong predictive performance even on real-world, noisy data.

Overall, this experiment shows that foundation models for time series forecasting can be successfully adapted to practical forecasting tasks and achieve competitive performance with limited fine-tuning.